In [2]:
%matplotlib inline
import seaborn as sns; 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import gridspec
import scipy
style.use('ggplot')
import datetime
from io import StringIO
import re
import numpy as np
from datetime import datetime

# List of all layers of all official docker images available for the Linux OS/amd64 by 23 March 2018
Debian have updated their images on 14 March

In [63]:
official_layers=pd.read_csv('../../docker/csv/layers_official_23MARCH.csv', sep=';', dtype=object, index_col=None,  error_bad_lines=False)
official_layers['level']=official_layers['level'].apply(int)
official_layers=official_layers.sort_values(['tag','slug','level'])
official_layers.head(2)

,slug,tag,created,layer,level
103061,ghost,0,2018-03-16T19:55:41.593708348Z,sha256:f2b6b4884fc8b2f1fcef843f92f7c82c9c149df...,1
103062,ghost,0,2018-03-16T19:55:41.593708348Z,sha256:4fb899b4df216ab398fb05547f86dc371db944c...,2


In [65]:
official_layers.sort_values('level', ascending=False, inplace=True)

In [66]:
official_layers = official_layers[['slug','level']]

In [28]:
community_layers=pd.read_csv('../../docker/csv/layers_community_march_12k.csv', sep=';', dtype=object, index_col=None,  error_bad_lines=False)
community_layers['level']=community_layers['level'].apply(int)
community_layers=community_layers.sort_values(['tag','slug','level'])
#community_layers.rename(columns={'tag':'name'}, inplace = True)
community_layers.head(2)

,slug,tag,created,layer,level
79718,devopscenter,0099ff.web,2014-08-27T02:44:47.123710403Z,sha256:a3ed95caeb02ffe68cdd9fd84406680ae93d633...,1
79719,devopscenter,0099ff.web,2014-08-27T02:44:47.123710403Z,sha256:23efb549476f5f10a40b3784758a807c0194d87...,2


In [29]:
# Let's drop duplicates
official_layers=official_layers.set_index(['slug','tag','level']).drop_duplicates(keep='first')
community_layers=community_layers.set_index(['slug','tag','level']).drop_duplicates(keep='first')

In [30]:
len(community_layers.query('slug == "007lva" and tag == "outletbox-rails"'))

19

In [34]:
community_layers = (community_layers.reset_index().set_index(['slug','tag'])
                       .drop(['level','created'], axis=1)
                       .groupby(['slug','tag'])
                       .apply(lambda x: x['layer'].sum())
                       .reset_index()
                       .rename(columns={0:'layer'})                       
                      )
community_layers.head(2)

,slug,tag,layer
0,007lva,outletbox-rails,sha256:d4bce7fd68df2e8bb04e317e7cb7899e981159a...
1,03192859189254,docker-magento-php-fpm-alpine,sha256:605ce1bd3f3164f2949a30501cc596f52a72de0...


In [35]:
len(community_layers.query('slug == "007lva" and tag == "outletbox-rails"').layer[0].split('sha256')) - 1 

19

In [36]:
len(community_layers[['slug','tag']].drop_duplicates()) == len(community_layers.drop_duplicates())

True

In [38]:
official_layers = (official_layers.reset_index().set_index(['slug','tag'])
                       .drop(['level','created'], axis=1)
                       .groupby(['slug','tag'])
                       .apply(lambda x: x['layer'].sum())
                       .reset_index()
                       .rename(columns={0:'layer'})                       
                      )
official_layers.head(2)

,slug,tag,layer
0,adminer,4,sha256:605ce1bd3f3164f2949a30501cc596f52a72de0...
1,adminer,4-fastcgi,sha256:605ce1bd3f3164f2949a30501cc596f52a72de0...


In [39]:
len(official_layers[['slug','tag']].drop_duplicates()) == len(official_layers)

True

In [40]:
len(official_layers)*len(community_layers)

74434410

In [41]:
official_layers.head()

,slug,tag,layer
0,adminer,4,sha256:605ce1bd3f3164f2949a30501cc596f52a72de0...
1,adminer,4-fastcgi,sha256:605ce1bd3f3164f2949a30501cc596f52a72de0...
2,adminer,4.2,sha256:7095154754192bfc2306f3b2b841ef82771b7ad...
3,adminer,4.2-fastcgi,sha256:7095154754192bfc2306f3b2b841ef82771b7ad...
4,adminer,4.3,sha256:605ce1bd3f3164f2949a30501cc596f52a72de0...


In [42]:
official_debian = official_layers.query('slug =="debian"')
official_debian.rename(columns={'slug':'slug_debian','tag':'tag_debian','layer':'layer_debian'}, inplace=True)
len(official_debian)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


131

In [43]:
official_layers.head(2)

,slug,tag,layer
0,adminer,4,sha256:605ce1bd3f3164f2949a30501cc596f52a72de0...
1,adminer,4-fastcgi,sha256:605ce1bd3f3164f2949a30501cc596f52a72de0...


In [44]:
official_debian.head(2)

,slug_debian,tag_debian,layer_debian
675,debian,6,sha256:069873d23334d65630bbe5e303ced0c68181b69...
676,debian,6.0.8,sha256:a3ed95caeb02ffe68cdd9fd84406680ae93d633...


In [45]:
len(official_layers)*len(official_layers)

79085449

In [46]:
official_layers['col_tmp']='1'
official_debian['col_tmp']='1'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
df_tmp = (official_layers
          .merge(official_debian,
                 on='col_tmp',
                 how = 'outer')
         ).drop('col_tmp',axis=1)
len(df_tmp)

1164983

In [48]:
df_tmp['relation'] = df_tmp.apply(lambda d: d['layer_debian'] in d['layer'], axis=1)

In [49]:
df_tmp.query('relation == True')[['slug','tag','layer']].set_index(['slug','tag']).drop_duplicates().size

2595

### Community now

In [50]:
community_layers['col_tmp']='1'
official_debian['col_tmp']='1'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [51]:
df_tmp_com = (community_layers
          .merge(official_debian,
                 on='col_tmp',
                 how = 'outer')
         ).drop('col_tmp',axis=1)
len(df_tmp_com)

1096470

In [52]:
df_tmp_com['relation'] = df_tmp_com.apply(lambda d: d['layer_debian'] in d['layer'], axis=1)

In [53]:
df_tmp_com.query('relation == True')[['slug','tag','layer']].set_index(['slug','tag']).drop_duplicates().size

1582